In [ ]:
# Compare all cause death between cats treated with daily RT (n=25 blue)) vs weekly RT (n=12 yellow)
# Compare incidence of systemic lymphoma between groups (blue = daily) vs yellow = weekly RT)
# compare
# compare whether survival between both groups differed between  those with and without steroid treatment- AB
# compare whether survival differed between groups with and without L-Asparaginase treatment before radiation treatment before radiation-  AE
# compare whether survival differed between groups with and without chemotherapy  treatment AFTER radiation treatment before radiation-  AG
# Compare whether overall survival between groups differed based upon group age- P


In [1]:

# py -3.11 -m pip install --upgrade openpyxl

import os, saspy, itertools, sys
import seaborn as sns
from tabulate import tabulate
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
today = datetime.today().strftime("%Y-%m-%d")
# now = datetime.now()

import matplotlib.pyplot as plt
from scipy.stats import bootstrap
from scipy.stats import pearsonr
from scipy.stats import mood


#Stats Import
from scipy.stats import wilcoxon 
from scipy.stats import shapiro 
from scipy.stats import kstest
# from numpy.random import seed
# from numpy.random import math
import pprint
import pingouin as pg
from pingouin import pairwise_ttests, pairwise_tukey, pairwise_gameshowell
import statsmodels.stats.multicomp as mc
from statsmodels.stats.multicomp import pairwise_tukeyhsd
from statsmodels.stats.multicomp import MultiComparison
import scipy.stats as stats
# from scipy import stats # Might need as is for ttest proportiality z or tdist
import statsmodels.api as sm
from bioinfokit.analys import stat
from statsmodels.formula.api import ols
from scipy.stats import wilcoxon
from scipy.stats import bootstrap
from statsmodels.stats.proportion import proportions_ztest
from scipy.stats import mannwhitneyu
from scipy.stats import norm

from lifelines.statistics import multivariate_logrank_test
from lifelines.statistics import pairwise_logrank_test
from lifelines import KaplanMeierFitter
from lifelines.utils import median_survival_times
from lifelines.utils import qth_survival_time
from lifelines import CoxPHFitter
from lifelines import KaplanMeierFitter
from lifelines.statistics import logrank_test

import io
from io import BytesIO
import base64


from operator import concat

sys.path.append(os.path.join(sys.path[0],'U:\lamb_consulting'))


# import cox, means, regressions

# from lamb_consulting import *

from lamb_consulting.means import means_desc, means_desc_class, aov
from lamb_consulting.freqs import freqs, chi_contingency, chi_oneway
# from lamb_consulting.export import export_to_Excel
# # from lamb_consulting.regressions import ols, regression_plot
from lamb_consulting.cox import cox
from lamb_consulting.kaplan_meier import km,km_plots


# For figure image to base 64 writing

def fig_to_base64(fig):
    img = io.BytesIO()
    fig.savefig(img, format='png',
                bbox_inches='tight')
    img.seek(0)

    return base64.b64encode(img.getvalue())

tpath = "U://Fox//Yeh//"
in_path = tpath + "//input//" 
output_path = tpath + "output//"
fn="yeh_infile_20230415"
extension='.xlsx'
file = in_path+fn+extension
file
yeh = pd.read_excel((file), sheet_name='data')

yeh.columns= yeh.columns= yeh.columns.str.lower()
yeh.columns.str.lower()

yeh.reset_index(inplace=True)
yeh = yeh.rename(columns = {'index':'my_id'})

yeh_col_l = yeh.columns.tolist()
print(yeh_col_l)
print("Code Location")
print(os.getcwd())

age_cond = [
    (yeh['age'] < 2900),
    (yeh['age'] >= 2900) & (yeh['age'] <= 4800),
    (yeh['age'] > 4800)
    ]

age_vals = ['age_l', 'age_m', 'age_h']
yeh['age_cat'] = np.select(age_cond, age_vals)
iv_cont_list = ["age", "dth_t"]
iv_cat_list = ["age_lt_10y", "weekly_trt", "breed_dsh", "steroid_therapy", "asparaginase", "chemo"]
# iv_cat_list = ["age_cat", "age_lt_10y", "weekly_trt", "breed_dsh", "steroid_therapy", "asparaginase", "chemo"]


today_file = datetime.today().strftime("%Y%m%d")
# print("Today's production date and time for file printing is : ", now.strftime("%Y%m%d"))



['my_id', 'my_id', 'patient_id', 'name', 'dob', 'age', 'age_y_fup', 'age_lt_10y', 'clinical_signs', 'radiation_treatment', 'radiation_treatment_end', 'systemic_disease_dt', 'date_of_death', 'date_last_reported_alive', 'last_dt', 'dth_t', 'dth_e', 'death_lymphoma', 'pred_start', 'pred_stop_date', 'elspar_given_date', 'time_to_progression', 'daily_treatments', 'weekly_treatments', 'weekly_trt', 'radiation_dose', 'systemic_lymphoma', 's', 'female', 'male', 'breed', 'breed_dsh', 'felv_neg', 'felv_pos', 'fiv_pos', 'steroid_therapy', 'asparaginase', 'chemo']
Code Location
\\192.168.68.75\ken\stats\Consulting\KEL\Fox\Yeh\code


# Single Strata Figure and IQR

In [3]:
# kmf = KaplanMeierFitter()
# t = yeh["dth_t"]
# # e = yeh["dth_e"]
# e = yeh["death_lymphoma"]

def single_km_strata(t,e,t_nm,e_nm):
    kmf = KaplanMeierFitter()
    kmf.fit(t, event_observed=e)
    kmf.plot_survival_function(at_risk_counts=True)
    plt.title('Survival function of political regimes')
    plt.ylabel("est. probability of survival $\hat{S}(t)$")
    plt.xlabel("time $t$")
    plt.tight_layout()
    print(qth_survival_time(0.75, kmf))
    print(median_survival_times(kmf))
    print(qth_survival_time(0.25, kmf))
    u_df = pd.DataFrame([qth_survival_time(0.75, kmf)], columns=['upper_75'])
    m_df = pd.DataFrame([median_survival_times(kmf)], columns=['median'])
    l_df = pd.DataFrame([qth_survival_time(0.25, kmf)], columns=['lower_25'])

    ul_in = u_df.merge(m_df, how='cross')
    ul_in = ul_in.merge(l_df, how='cross')
    ul_in['t'] = t_nm
    ul_in['e'] = e_nm
    print(ul_in)
    # plt.savefig(output_path + "km_plt_" + "dth_t" +"_" + "dth_e" +".png", dpi=600)
    plt.savefig(output_path + "km_plt_" + t_nm +"_" + e_nm +".png", dpi=600)
    plt.clf() #clear the plot after saving
    return ul_in


single_dth_e = single_km_strata(yeh["dth_t"],yeh["dth_e"],"dth_t","dth_e")
single_death_lymphoma = single_km_strata(yeh["dth_t"],yeh["death_lymphoma"],"dth_t","death_lymphoma")

single_km_all = pd.concat([single_dth_e,single_death_lymphoma])



c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\f

105.0
203.0
1020.0
   upper_75  median  lower_25      t      e
0     105.0   203.0    1020.0  dth_t  dth_e


c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\fitters\__init__.py:271: ApproximationWarning: Approximating using `survival_function_`. To increase accuracy, try using or increasing the resolution of the timeline kwarg in `.fit(..., timeline=timeline)`.

  warnings.warn(
c:\Users\Ken\AppData\Local\Programs\Python\Python311\Lib\site-packages\lifelines\f

112.0
667.0
inf
   upper_75  median  lower_25      t               e
0     112.0   667.0       inf  dth_t  death_lymphoma


<Figure size 640x480 with 0 Axes>

# Get Median Survival and IQR per strata

In [5]:

km_strata_iqr = pd.DataFrame()

for i in range(0,len(iv_cat_list)):
    iqr_dth = km(yeh,"yeh",iv_cat_list[i],"dth_t","dth_e",i)
    km_strata_iqr = pd.concat([km_strata_iqr,iqr_dth])
    iqr_lymphoma = km(yeh,"yeh",iv_cat_list[i],"dth_t","death_lymphoma",i)
    km_strata_iqr = pd.concat([km_strata_iqr,iqr_lymphoma])


print(type(km_strata_iqr))
display(km_strata_iqr)


<class 'pandas.core.frame.DataFrame'>


,pop,research_q,iv,dv,event,mod_num,percent,estimate,lowerLimit,upperLimit,iv_level
0,yeh,0,age_lt_10y,dth_t,dth_e,0,75,865.0,160.0,.,1
0,yeh,0,age_lt_10y,dth_t,dth_e,0,50,187.0,78.0,.,1
0,yeh,0,age_lt_10y,dth_t,dth_e,0,25,143.0,78.0,.,1
0,yeh,0,age_lt_10y,dth_t,dth_e,0,75,1020.0,207.0,2373.0,0
0,yeh,0,age_lt_10y,dth_t,dth_e,0,50,205.0,69.0,667.0,0
...,...,...,...,...,...,...,...,...,...,...,...
0,yeh,5,chemo,dth_t,death_lymphoma,0,50,183.0,89.0,1020.0,1
0,yeh,5,chemo,dth_t,death_lymphoma,0,25,89.0,52.0,1020.0,1
0,yeh,5,chemo,dth_t,death_lymphoma,0,75,.,.,.,0
0,yeh,5,chemo,dth_t,death_lymphoma,0,50,.,160.0,.,0


# Cox Model Single Variable

In [6]:
cox_vars = ["age"]
cox_single = pd.DataFrame()

for i in range(0,len(cox_vars)):
    # coxdf = coxdf.append(cox(yeh,cox_vars[i],"dth_t","dth_e"),ignore_index=True)
    my_cox = cox(yeh,cox_vars[i],"dth_t","dth_e")
    cox_single = pd.concat([cox_single,my_cox])

display(cox_single.head())

,iv,dv,estimate,stderr,p_value,hazardRatio,hrlowercl,hruppercl
1,age,dth_e,0.000085,0.000163,0.601467,1.000085,0.999765,1.000405


# Pairwise log rank comparisons

In [7]:
# Use pairwise_logrank_test for everything since it works with single and multiple pairwise... will merge on cross merge
# my_prw = pairwise_logrank_test(yeh['dth_t'], yeh['age_cat'], yeh['dth_e'])

# death_ty = 'dth_e'
# death_ty = 'death_lymphoma'

def log_pairs(death_ty):
    pair_data = pd.DataFrame()
    for i in iv_cat_list: 
        # print(len(iv_cat_list)) 
        # print(i)      
        my_prw = pairwise_logrank_test(yeh['dth_t'], yeh[i], yeh[death_ty])
        my_nm = my_prw.name
        my_p = my_prw.p_value.tolist()
        zipme = list(zip(my_nm, my_p))
        my_pairs = pd.DataFrame(zipme, columns = ['comparisons', 'p_val'])
        # print(type(my_pairs))
        my_pairs['iv'] =  i
        my_pairs['dv'] =  death_ty
        pair_data = pd.concat([pair_data,my_pairs])
        # display(pair_data.head())
    return pair_data



log_death_lymphoma = log_pairs('death_lymphoma')
log_dth_e = log_pairs('dth_e')

log_all = pd.concat([log_death_lymphoma,log_dth_e])
log_all.head(20)

,comparisons,p_val,iv,dv
0,"(0, 1)",0.881722,age_lt_10y,death_lymphoma
0,"(0, 1)",0.023809,weekly_trt,death_lymphoma
0,"(0, 1)",0.171696,breed_dsh,death_lymphoma
0,"(0, 1)",0.053174,steroid_therapy,death_lymphoma
0,"(0, 1)",0.058829,asparaginase,death_lymphoma
0,"(0, 1)",0.026684,chemo,death_lymphoma
0,"(0, 1)",0.552211,age_lt_10y,dth_e
0,"(0, 1)",0.004927,weekly_trt,dth_e
0,"(0, 1)",0.369073,breed_dsh,dth_e
0,"(0, 1)",0.079146,steroid_therapy,dth_e


# Get plots to PNG

In [3]:

for i in iv_cat_list:
    km_plots(yeh,i,'dth_t','death_lymphoma',i,False,output_path)
    km_plots(yeh,i,'dth_t','dth_e',i,False,output_path)


<Figure size 640x480 with 0 Axes>

# Stack Each Strata for Comprehensive Duration/Survivorship plot at end

In [8]:
# cph = CoxPHFitter().fit(df, 'T', 'E')
# cph.print_summary()

# lt_df = pd.DataFrame()

# ax = plt.subplot(111)
def km_stacker(dth_t,dth_e):
    lt_df = pd.DataFrame()
    kmf = KaplanMeierFitter()
    for i in iv_cat_list:
        for name, grouped_df in yeh.groupby(i):
            kmf.fit(grouped_df[dth_t], grouped_df[dth_e], label=name)
            # kmf.plot_survival_function(ax=ax)
            # b = kmf.confidence_interval_ # Include this if we want the CI
            my_lt = kmf.survival_function_ 
            my_lt.rename(columns={my_lt.columns[0]: 's'}, inplace=True)
            my_lt["iv"] = i
            my_lt['level'] = name
            my_lt['dth_t'] =  dth_t
            my_lt['dth_e'] =  dth_e
            lt_df = pd.concat([lt_df,my_lt])
    return lt_df

stack_dth_e = km_stacker('dth_t', 'dth_e')
stack_death_lymphoma = km_stacker('dth_t', 'death_lymphoma')
stacked_lt = pd.concat([stack_dth_e,stack_death_lymphoma])
stacked_lt.head(25)

,s,iv,level,dth_t,dth_e
timeline,,,,,
0.0,1.000000,age_lt_10y,0,dth_t,dth_e
30.0,0.954545,age_lt_10y,0,dth_t,dth_e
37.0,0.909091,age_lt_10y,0,dth_t,dth_e
52.0,0.863636,age_lt_10y,0,dth_t,dth_e
61.0,0.772727,age_lt_10y,0,dth_t,dth_e
69.0,0.727273,age_lt_10y,0,dth_t,dth_e
89.0,0.681818,age_lt_10y,0,dth_t,dth_e
112.0,0.636364,age_lt_10y,0,dth_t,dth_e
145.0,0.590909,age_lt_10y,0,dth_t,dth_e


# Start Groupings for mean, std, and other descriptive statistics



In [11]:

m_desc_df = pd.DataFrame()
m_class_df = pd.DataFrame()

for i in iv_cont_list:
        desc_tmp = means_desc(yeh, i,"yeh")
        m_desc_df = pd.concat([m_desc_df,desc_tmp])


# Deliberately changing class to desc for appending for stacking purposes
for i in iv_cat_list:
    for j in iv_cont_list:
        cat_missing = yeh[i].isnull().sum()
        miss_stat_ca = pd.DataFrame([cat_missing], columns=['nmiss_cat'])
        cont_missing = yeh[j].isnull().sum()
        miss_stat_co = pd.DataFrame([cont_missing], columns=['nmiss_cont'])
        my_describe = yeh.groupby([i])[j].describe()        
        # display(my_describe.head())
        desc_class_tmp = my_describe.merge(miss_stat_ca, how='cross')
        desc_class_tmp = desc_class_tmp.merge(miss_stat_co, how='cross')
        desc_class_tmp['iv'] = i
        desc_class_tmp['dv'] = j
        desc_class_tmp.reset_index(inplace=True)
        desc_class_tmp = desc_class_tmp.rename(columns = {'index':'iv_level'})
        m_class_df = pd.concat([m_class_df,desc_class_tmp])


print("m_desc_df")
display(m_desc_df.head())
print("m_class_df")
display(m_class_df.head(25))

m_desc_df


,df_used,cont_var,nmiss,n,mean,median,std,min,max,p25,p75
0,yeh,age,0,34.0,3922.647059,3924.0,1460.104519,1144.0,6340.0,2970.50,4896.0
0,yeh,dth_t,0,34.0,418.705882,185.0,502.183507,30.0,2373.0,80.75,661.5


m_class_df


,iv_level,count,mean,std,min,25%,50%,75%,max,nmiss_cat,nmiss_cont,iv,dv
0,0,22.0,4790.818182,866.924539,3729.0,3962.25,4722.5,5390.75,6340.0,0,0,age_lt_10y,age
1,1,12.0,2331.000000,830.563992,1144.0,1733.25,2551.0,2970.50,3379.0,0,0,age_lt_10y,age
0,0,22.0,445.409091,555.085807,30.0,74.00,205.0,641.75,2373.0,0,0,age_lt_10y,dth_t
1,1,12.0,369.750000,405.332664,36.0,98.25,151.5,701.75,1218.0,0,0,age_lt_10y,dth_t
0,0,22.0,3440.045455,1412.594117,1144.0,2678.75,3554.0,4351.00,6046.0,0,0,weekly_trt,age
1,1,12.0,4807.416667,1125.380942,2432.0,3996.50,4948.5,5447.50,6340.0,0,0,weekly_trt,age
0,0,22.0,518.818182,577.114632,36.0,105.25,277.5,809.75,2373.0,0,0,weekly_trt,dth_t
1,1,12.0,235.166667,252.836934,30.0,58.75,127.5,305.75,667.0,0,0,weekly_trt,dth_t
0,0,12.0,3910.500000,1725.019921,1146.0,3336.00,3984.5,4823.25,6340.0,0,0,breed_dsh,age
1,1,22.0,3929.272727,1338.398196,1144.0,2970.50,3865.0,4987.25,6046.0,0,0,breed_dsh,age


# Cross Tabular Frequency

In [10]:



my_list = iv_cat_list.copy()

iter_list = list(itertools.combinations(my_list, 2))


print(len(iter_list))
cross_cat = pd.DataFrame()

ct = 1

if ct ==1:
    f = open("df_ct_" + today_file + ".html",'w')

my_len = (len(iter_list))
for i in range(0, len(iter_list)):
    # print(i)
    res = [iter_list[i]]
    
    a = res[0] 
    v1,v2 = a
    # cross_tmp, propz_df = chi_contingency(sch,v1,v2,"1","chi_cont","sch")
    cross_tmp, propz_df = chi_contingency(yeh,v1,v2,"1","chi_cont","yeh")
    # cross_cat = cross_cat.append(cross_tmp)
    cross_cat = pd.concat([cross_cat,cross_tmp])
    if ct ==1:
        my_html = propz_df.to_html()
        f.write("The Cross Tabular variables are {} and {}.".format(v1, v2))
        f.write(my_html)

if ct ==1:
    f.close()

print("cross_cat")
print(cross_cat.tail(5))



15
cross_cat
  df_used r_q_n  analysis research_q                       Table  Frequency   
0     yeh     1  chi_cont          1  Table asparaginase * chemo         10  \
1     yeh     1  chi_cont          1  Table asparaginase * chemo          7   
2     yeh     1  chi_cont          1  Table asparaginase * chemo          6   
3     yeh     1  chi_cont          1  Table asparaginase * chemo         11   
1     yeh     1  chi_cont          1  Table asparaginase * chemo         34   

      Percent Missing iv1 iv2  
0   29.411765       .   0   0  
1   20.588235       .   0   1  
2   17.647059       .   1   0  
3   32.352941       .   1   1  
1  100.000000       0   .   .  


In [12]:
 
today_file = datetime.today().strftime("%Y%m%d")
# print("Today's production date and time for file printing is : ", now.strftime("%Y%m%d"))


writer = pd.ExcelWriter(output_path + 'yeh_output_l_' + today_file + '.xlsx', engine = 'xlsxwriter')

# Data In
yeh.to_excel(writer,sheet_name='data_in')

# General Stats to one sheet
single_km_all.to_excel(writer, sheet_name='KM', index=False, startcol=0,startrow=0) # Population Single Strata
log_all.to_excel(writer, sheet_name='KM', index=False, startcol=7,startrow=0)
km_strata_iqr.to_excel(writer, sheet_name='KM_Strata_IQR') # Population Single Strata
stacked_lt.to_excel(writer, sheet_name='LT_All_S_Times') # All Times all strata
cox_single.to_excel(writer, sheet_name='Cox_Uni') # All Times all strata


m_desc_df.to_excel(writer,sheet_name='desc')
m_class_df.to_excel(writer,sheet_name='desc_class')
cross_cat.to_excel(writer,sheet_name='cross_cat')

writer.close()

# Working test code for IQR, now in general library

In [ ]:

time="dth_t"
event="dth_e"
j="weekly_trt"
var="weekly_trt"
r_q = 1
pop = "yeh"
ds = 'yeh'

levels = pd.unique(yeh[j])
kms = []
temp = []
estimates = []
lower = []
upper = []
df = pd.DataFrame()
for i in range(0,len(levels)):
        kms.append(KaplanMeierFitter())
        temp.append(yeh[yeh[var]==levels[i]])        
        kms[i].fit(durations=temp[i][time],event_observed=temp[i][event],label=j+"="+str(levels[i]))
        estimates.append(get_KM_estimates(kms[i].survival_function_,var+"="+str(levels[i]),"timeline"))
        lower.append(get_KM_estimates(kms[i].confidence_interval_,var+"="+str(levels[i])+"_lower_0.95","index"))
        upper.append(get_KM_estimates(kms[i].confidence_interval_,var+"="+str(levels[i])+"_upper_0.95","index"))
for i in range(0,len(levels)-1):
        for j in range(1, len(levels)):
                if (i != j):
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"75", "estimate":estimates[i][2], "lowerLimit":lower[i][2], "upperLimit":upper[i][2], "iv_level":levels[i]},orient='index').T
                        df = pd.concat([df,me_in])
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"50", "estimate":estimates[i][1], "lowerLimit":lower[i][1], "upperLimit":upper[i][1], "iv_level":levels[i]},orient='index').T
                        df = pd.concat([df,me_in])
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"25", "estimate":estimates[i][0], "lowerLimit":lower[i][0], "upperLimit":upper[i][1], "iv_level":levels[i]},orient='index').T
                        df = pd.concat([df,me_in])
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"75", "estimate":estimates[j][2], "lowerLimit":lower[j][2], "upperLimit":upper[j][2], "iv_level":levels[j]},orient='index').T
                        df = pd.concat([df,me_in])
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"50", "estimate":estimates[j][1], "lowerLimit":lower[j][1], "upperLimit":upper[j][1], "iv_level":levels[j]},orient='index').T
                        df = pd.concat([df,me_in])
                        me_in = pd.DataFrame.from_dict({"pop":pop,"research_q":r_q, "iv":var, "dv":time, "mod_num":0, "percent":"25", "estimate":estimates[j][0], "lowerLimit":lower[j][0], "upperLimit":upper[j][1], "iv_level":levels[j]},orient='index').T
                        df = pd.concat([df,me_in])

                
print(i)
print(j)
print(estimates)
print(lower)
print(upper)
# print(a)
# print(type(a))
display(df.head(50))


# df = df.append({"pop":pop,"research_q":r_q, "iv":var,"dv":time,"mod_num":0,"test":"Log-Rank","P_value":logrank(temp[i],temp[j],time,event),"percent":"75","estimate":estimates[i][2],"lowerLimit":lower[i][2],"upperLimit":upper[i][2],"iv_level":levels[i]},ignore_index=True)




coxph  = CoxPHFitter()
ds = yeh.dropna(subset=['age'])
ds = ds[['dth_t', 'dth_e', 'age', 'breed_dsh']]
# coxph .fit(ds, duration_col = 'dth_t', event_col = 'dth_e', formula="age + breed_dsh + age * breed_dsh") # "fin + wexp + age * prio" ['age', 'breed_dsh']
coxph .fit(ds, duration_col = 'dth_t', event_col = 'dth_e', formula="age") # "fin + wexp + age * prio" ['age', 'breed_dsh']
coxph .print_summary()
coxph .check_assumptions(ds)
coxph.plot()
